In [ ]:
import pandas as pd

In [ ]:
from google.colab import files 
  
  
uploaded = files.upload()

Saving Real_Combine.csv to Real_Combine.csv


In [ ]:
import pandas as pd 
import io 
  
df = pd.read_csv(io.BytesIO(uploaded['Real_Combine.csv'])) 

In [ ]:
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [ ]:
!pip install -U keras-tuner

     |████████████████████████████████| 71kB 3.7MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp36-none-any.whl size=78937 sha256=cbb422f6c3ad7a6ce9822d062e3e3cd180362ce8d8d70e9d939cfca743dd941a
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=5701cf43b222370a5d6209b0be19f285f2038d0d6649e675aeaabd8461cf0514
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
from tensorflow.python.keras.metrics import Metric


In [ ]:
X=df.iloc[:,:-1]
X.head(5)

,T,TM,Tm,SLP,H,VV,V,VM
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2


In [ ]:
Y=df.iloc[:,-1]
Y.head()

0    219.720833
1    182.187500
2    154.037500
3    223.208333
4    200.645833
Name: PM 2.5, dtype: float64

In [ ]:
def build_model(hp):
    model=keras.Sequential()
    for i in range(hp.Int('num_layers',2,20)):
        model.add(layers.Dense(units=hp.Int('units_'+str(i),min_value=8,max_value=512,step=32),activation="relu"))
    model.add(layers.Dense(1,activation='linear'))
    #linear regression problem of predicting air quality
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',[1e-2,1e-3,1e-4])),loss='mean_absolute_error',metrics=['mean_absolute_error'])
    return model  

In [ ]:
tuner=RandomSearch(build_model,objective='val_mean_absolute_error',
                  max_trials=5,executions_per_trial=3,
                  directory='project',
                  project_name='AQI')

In [ ]:

tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 8, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 8, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,Y,random_state=0,test_size=0.2)

In [33]:
tuner.search(X_train,y_train,epochs=5,
            validation_data=(X_test,y_test))

INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary()

Results summary
Results in project/AQI
Showing 10 best trials
Objective(name='val_mean_absolute_error', direction='min')
Trial summary
Hyperparameters:
num_layers: 9
units_0: 456
units_1: 40
learning_rate: 0.01
units_2: 8
units_3: 8
units_4: 8
units_5: 8
units_6: 8
units_7: 8
units_8: 8
Score: 50.71690241495768
Trial summary
Hyperparameters:
num_layers: 18
units_0: 424
units_1: 232
learning_rate: 0.001
units_2: 40
units_3: 200
units_4: 72
units_5: 104
units_6: 360
units_7: 392
units_8: 488
units_9: 8
units_10: 296
units_11: 8
units_12: 424
units_13: 392
units_14: 360
units_15: 488
units_16: 296
units_17: 8
Score: 57.76760228474935
Trial summary
Hyperparameters:
num_layers: 16
units_0: 104
units_1: 168
learning_rate: 0.001
units_2: 168
units_3: 72
units_4: 72
units_5: 40
units_6: 104
units_7: 232
units_8: 232
units_9: 392
units_10: 392
units_11: 456
units_12: 136
units_13: 104
units_14: 392
units_15: 40
units_16: 40
Score: 60.90863291422526
Trial summary
Hyperparameters:
num_layers: 17
